In [1]:
%run DatasetManager.ipynb
%run TransformManager.ipynb
%run ValidationManager.ipynb

In [2]:
import os.path as osp
from pathlib import Path
from fastai.vision.data import DataBlock, ImageBlock, MaskBlock
from fastai.data.transforms import RandomSplitter
from fastcore.foundation import L
from functools import partial

In [3]:
class DataLoaderManager():
    def __init__(self, dataset, transforms = TransformManagerFastai(), validation = ValidationManagerTrainValTest(),
                 batch_size = 4):
        """
        Description:
        Builds a fastai DataBlock for each fold in validation.
        
        Parameters:
        dataset (DatasetManager): the dataset where the data is.
        transforms (TransformManagerFastai): the transforms applied to the data.
        validation (ValidationManager, ValidationManagerTrainValTest): the way to split the data.
        batch_size (int, 4): the batch size for each learning iteration.

        Returns:
        dbm (DataBlockManager): the built DataBlockManager.
        """
        self.dataset_ = dataset
        self.transforms_ = transforms
        self.validation_ = validation
        self.batch_size_ = batch_size
    
    def __get_files_from__(source, files, self):
        """
        Description:
        Gets all the paths saved in the files.

        Parameters:
        source (str): ignored. Just for backwards compatibility.
        files (List[str]): the paths to the files.
        self (DataLoaderManager): the object itself.

        Returns:
        files (L): a L list-like object with all the paths to each image in the files.
        """
        l = []
        for file in files: 
            with open(file, "r") as f:
                l.extend([
                    Path(osp.join(self.dataset_.root_dir_, "images", path + self.dataset_.img_suffix_))
                    for path in f.read().split("\n")[:-1]
                ])

        return L(l).unique()

    def __splits_items__(items, val_file):
        """
        Description:
        Splits items in train and validation lists.

        Parameters:
        items (L(Path)): the L list-like with all the data.
        val_file (str): path-like. The path to the validation file.

        Returns:
        splits (Tuple(L, L)): a tuple with the train and val splits.
        """
        with open(val_file, "r") as f:
            val_size = len(f.read().split("\n")[:-1])

        train = range(len(items) - val_size)
        val = range(len(train), len(items))
        return L(train), L(val)
    
    def __get_mask__(self, img):
        """
        Description:
        Gets the absolute path for the mask of the image.
        
        Parameters:
        img (str): the path to the image file.
        
        Returns:
        path (str): path-like str with the absolute path to the mask.
        """
        return self.dataset_.img_map_(osp.basename(img))

    def get_dataloaders(self):
        """
        Description:
        Gets all the datablocks for each dataset's fold.

        Parameters:
        None.
        
        Returns:
        datablocks (dict): the dict with the split architecture and the DataBlocks bounded.
        """
        split_result = self.validation_.split(self.dataset_)
        dataloaders = {}
        for fold, path in split_result.items():
            # if the fold is for testing
            if fold == "test":
                # the test training
                db = DataBlock(blocks = (ImageBlock, MaskBlock(self.dataset_.class_names_)),
                               get_items = partial(DataLoaderManager.__get_files_from__, files = files, self = self),
                               get_y = partial(self.__get_mask__),
                               splitter = RandomSplitter(valid_pct = 0.1),
                               item_tfms = self.transforms_.get_pipeline()
                 )
                dataloaders.update({fold: db.dataloaders(None, bs = self.batch_size_)})
                
                # the test validation
                files.append(osp.join(self.dataset_.root_dir_, "splits", "test.txt"))
                db = DataBlock(blocks = (ImageBlock, MaskBlock(self.dataset_.class_names_)),
                               get_items = partial(DataLoaderManager.__get_files_from__, files = files, self = self),
                               get_y = partial(self.__get_mask__),
                               splitter = partial(DataLoaderManager.__splits_items__, val_file = files[2]),
                               item_tfms = self.transforms_.get_pipeline()
                 )
                dataloaders.update({"validation": db.dataloaders(None, bs = self.batch_size_)})
            
            # if the fold is for training
            else:
                files = [
                    osp.join(self.dataset_.root_dir_, "splits", path["train"]),
                    osp.join(self.dataset_.root_dir_, "splits", path["val"])
                ]
                db = DataBlock(blocks = (ImageBlock, MaskBlock(self.dataset_.class_names_)),
                               get_items = partial(DataLoaderManager.__get_files_from__, files = files, self = self),
                               get_y = partial(self.__get_mask__),
                               splitter = partial(DataLoaderManager.__splits_items__, val_file = files[1]),
                               item_tfms = self.transforms_.get_pipeline()
                 )

                # updates the dataloaders
                dataloaders.update({fold: db.dataloaders(None, bs = self.batch_size_)})

        return dataloaders